In [ ]:
#Question 5

import numpy as np

S0 = 80
K = 80
r = 0.055
T = 0.25
kappa = 1.85
theta = 0.045
v0 = 0.032
xi = 0.35
rho = -0.30
steps = 200
Npaths = 50000

def generate_heston_paths(S0, T, r, kappa, theta, v0, rho, xi, steps, Npaths):
    dt = T / steps
    prices = np.zeros((Npaths, steps))
    S_t = np.full(Npaths, S0)
    v_t = np.full(Npaths, v0)
    for t in range(steps):
        cov_matrix = np.array([[1, rho], [rho, 1]])
        dW = np.random.multivariate_normal([0,0], cov_matrix, Npaths)*np.sqrt(dt)
        v_t = np.abs(v_t + kappa*(theta - v_t)*dt + xi*np.sqrt(v_t)*dW[:,1])
        S_t = S_t * np.exp((r - 0.5*v_t)*dt + np.sqrt(v_t)*dW[:,0])
        prices[:, t] = S_t
    return prices

prices = generate_heston_paths(S0, T, r, kappa, theta, v0, rho, xi, steps, Npaths)

discount_factor = np.exp(-r*T)
call_payoffs_q5 = np.maximum(prices[:,-1] - K, 0)
put_payoffs_q5 = np.maximum(K - prices[:,-1], 0)
call_price_q5 = discount_factor * np.mean(call_payoffs_q5)
put_price_q5 = discount_factor * np.mean(put_payoffs_q5)

call_price_q5, put_price_q5

(np.float64(2.8353989830407293), np.float64(2.8293818787304734))

In [ ]:
#Question 6

rho = -0.70

prices = generate_heston_paths(S0, T, r, kappa, theta, v0, rho, xi, steps, Npaths)

call_payoffs_q6 = np.maximum(prices[:,-1] - K, 0)
put_payoffs_q6 = np.maximum(K - prices[:,-1], 0)
call_price_q6 = discount_factor * np.mean(call_payoffs_q6)
put_price_q6 = discount_factor * np.mean(put_payoffs_q6)

call_price_q6, put_price_q6

(np.float64(2.092975786887371), np.float64(3.462382902869673))

In [ ]:
baseline_prices = {
    -0.30: {'call': call_price_q5, 'put': put_price_q5},
    -0.70: {'call': call_price_q6, 'put': put_price_q6},
}

def get_option_price(S0, rho):
    prices = generate_heston_paths(S0, T, r, kappa, theta, v0, rho, xi, steps, Npaths)
    call_payoffs = np.maximum(prices[:, -1] - K, 0)
    put_payoffs = np.maximum(K - prices[:, -1], 0)
    call_price = discount_factor * np.mean(call_payoffs)
    put_price = discount_factor * np.mean(put_payoffs)
    return call_price, put_price

def calculate_greeks_corrected(S0, rho, baseline_call, baseline_put, h=1.2):
    C_up, P_up = get_option_price(S0 + h, rho)
    C_down, P_down = get_option_price(S0 - h, rho)

    delta_call = (C_up - C_down) / (2 * h)
    delta_put = (P_up - P_down) / (2 * h)
    gamma_call = (C_up - 2 * baseline_call + C_down) / (h ** 2)
    gamma_put = (P_up - 2 * baseline_put + P_down) / (h ** 2)

    return {'call': {'delta': delta_call, 'gamma': gamma_call},
            'put': {'delta': delta_put, 'gamma': gamma_put}}

call_price_q5, put_price_q5 = baseline_prices[-0.30]['call'], baseline_prices[-0.30]['put']
call_price_q6, put_price_q6 = baseline_prices[-0.70]['call'], baseline_prices[-0.70]['put']

greeks_rho_30 = calculate_greeks_corrected(S0, -0.30, call_price_q5, put_price_q5)
greeks_rho_70 = calculate_greeks_corrected(S0, -0.70, call_price_q6, put_price_q6)

greeks_rho_30, greeks_rho_70

({'call': {'delta': np.float64(0.5362079133150529),
   'gamma': np.float64(0.08351521166368911)},
  'put': {'delta': np.float64(-0.45378501485224415),
   'gamma': np.float64(0.07522444241291153)}},
 {'call': {'delta': np.float64(0.4766336751778545),
   'gamma': np.float64(0.0862932163426069)},
  'put': {'delta': np.float64(-0.47512420596553084),
   'gamma': np.float64(0.02861590800133065)}})

In [ ]:
from sklearn.linear_model import LinearRegression

def generate_heston_paths(S0, T, r, kappa, theta, v0, rho, xi, steps, Npaths):
    dt = T / steps
    prices = np.zeros((Npaths, steps+1))
    volatilities = np.zeros((Npaths, steps+1))
    S_t = np.full(Npaths, S0)
    v_t = np.full(Npaths, v0)
    prices[:, 0] = S_t
    volatilities[:, 0] = v_t
    for t in range(1, steps+1):
        cov = np.array([[1, rho], [rho, 1]])
        W = np.random.multivariate_normal([0, 0], cov, size=Npaths) * np.sqrt(dt)
        v_t = np.abs(v_t + kappa * (theta - v_t) * dt + xi * np.sqrt(v_t) * W[:,1])
        S_t = S_t * np.exp((r - 0.5 * v_t) * dt + np.sqrt(v_t) * W[:,0])
        prices[:, t] = S_t
        volatilities[:, t] = v_t
    return prices, volatilities

def american_call_lsm(S0, K, r, T, kappa, theta, v0, rho, xi, steps, Npaths):
    dt = T / steps
    discount = np.exp(-r * dt)

    prices, _ = generate_heston_paths(S0, T, r, kappa, theta, v0, rho, xi, steps, Npaths)
    payoffs = np.maximum(prices[:, -1] - K, 0)

    cash_flows = payoffs.copy()

    for t in range(steps-1, 0, -1):
        itm_indices = np.where(prices[:, t] > K)[0]
        if len(itm_indices) == 0:
            continue
        X = prices[itm_indices, t].reshape(-1, 1)
        Y = cash_flows[itm_indices] * discount

        X_basis = np.hstack([np.ones_like(X), X, X**2])
        model = LinearRegression().fit(X_basis, Y)
        continuation_values = model.predict(X_basis)

        exercise_values = prices[itm_indices, t] - K

        exercise_now = exercise_values > continuation_values
        cash_flows[itm_indices[exercise_now]] = exercise_values[exercise_now]
        cash_flows[itm_indices] = cash_flows[itm_indices] * discount

    option_price = np.mean(cash_flows) * discount
    return option_price

def calculate_american_call_greeks(S0, K, r, T, kappa, theta, v0, rho, xi, steps, Npaths, h=1.2):
    price_0 = american_call_lsm(S0, K, r, T, kappa, theta, v0, rho, xi, steps, Npaths)
    price_up = american_call_lsm(S0 + h, K, r, T, kappa, theta, v0, rho, xi, steps, Npaths)
    price_down = american_call_lsm(S0 - h, K, r, T, kappa, theta, v0, rho, xi, steps, Npaths)

    delta = (price_up - price_down) / (2 * h)
    gamma = (price_up - 2 * price_0 + price_down) / (h**2)

    return {'price': price_0, 'delta': delta, 'gamma': gamma}

S0 = 80
K = 80
r = 0.055
T = 0.25
kappa = 1.85
theta = 0.045
v0 = 0.032
xi = 0.35
rho = -0.30
steps = 200
Npaths = 50000
h = 0.5

results = calculate_american_call_greeks(S0, K, r, T, kappa, theta, v0, rho, xi, steps, Npaths, h)
print(f"American Call Option Price: {results['price']}")
print(f"Delta: {results['delta']}")
print(f"Gamma: {results['gamma']}")

American Call Option Price: 2.863216147209768
Delta: 0.5563840898008086
Gamma: 0.16524426003749504


In [1]:
#Question 8
import math, numpy as np, pandas as pd
from math import exp, sqrt, log
np.random.seed(12345)

S0 = 80.0
K = 80.0   # ATM
r = 0.055
sigma = 0.35
T = 0.25  # 3 months
# Merton jump params
lam = 0.75        # jump intensity (lambda)
mu_j = -0.5       # mean of log jump size
delta = 0.228     # std dev of log jump size

# Simulation parameters
Nsim = 100000     # number of Monte Carlo paths
Nsteps = 50       # time steps for discretization
dt = T / Nsteps

# Precompute
kappa = math.exp(mu_j + 0.5 * delta**2) - 1.0  # E[J-1]
mu_r = r - lam * kappa - 0.5 * sigma**2       # adjusted drift for diffusion part

discount = math.exp(-r * T)

# simulate
S = np.full(Nsim, S0, dtype=np.float64)

sqrt_dt = math.sqrt(dt)
lam_dt = lam * dt

for step in range(Nsteps):
    # diffusion multiplier
    Z = np.random.normal(size=Nsim)
    diffusion = np.exp(mu_r * dt + sigma * sqrt_dt * Z)
    # jump counts for this step: Poisson(λ dt)
    Nj = np.random.poisson(lam_dt, size=Nsim)
    # jump multiplier: product of Nj iid lognormal jumps
    jump_mult = np.ones(Nsim, dtype=np.float64)
    idx = Nj > 0
    if np.any(idx):
        Nj_pos = Nj[idx]
        W = np.random.normal(size=Nj_pos.size)
        jump_mult[idx] = np.exp(mu_j * Nj_pos + delta * np.sqrt(Nj_pos) * W)
    # update S
    S *= diffusion * jump_mult

# payoffs
call_payoffs = np.maximum(S - K, 0.0)
put_payoffs  = np.maximum(K - S, 0.0)

# discounted prices and standard errors
call_price = discount * call_payoffs.mean()
put_price  = discount * put_payoffs.mean()
call_se = discount * call_payoffs.std(ddof=1) / math.sqrt(Nsim)
put_se  = discount * put_payoffs.std(ddof=1) / math.sqrt(Nsim)

# 95% CI
z = 1.96
call_ci = (call_price - z*call_se, call_price + z*call_se)
put_ci  = (put_price  - z*put_se,  put_price  + z*put_se)

# Black-Scholes reference (no jumps)
from math import erf
def std_norm_cdf(x):
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))
def bs_call_price(S, K, r, sigma, T):
    if T == 0:
        return max(S-K,0)
    d1 = (math.log(S/K) + (r + 0.5*sigma*sigma)*T) / (sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    return S * std_norm_cdf(d1) - K * math.exp(-r*T) * std_norm_cdf(d2)
def bs_put_price(S,K,r,sigma,T):
    if T == 0:
        return max(K-S,0)
    d1 = (math.log(S/K) + (r + 0.5*sigma*sigma)*T) / (sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    return K * math.exp(-r*T) * std_norm_cdf(-d2) - S * std_norm_cdf(-d1)

call_bs = bs_call_price(S0,K,r,sigma,T)
put_bs  = bs_put_price(S0,K,r,sigma,T)

print("Merton (MC) Call:", call_price, "SE:", call_se, "95% CI:", call_ci)
print("Merton (MC) Put: ", put_price,  "SE:", put_se,  "95% CI:", put_ci)
print("Black-Scholes Call:", call_bs, "Put:", put_bs)


Merton (MC) Call: 8.294928561985028 SE: 0.03621707702146869 95% CI: (np.float64(8.22394309102295), np.float64(8.365914032947106))
Merton (MC) Put:  7.155371764010456 SE: 0.03866861504831044 95% CI: (np.float64(7.079581278515768), np.float64(7.231162249505145))
Black-Scholes Call: 6.103270146060531 Put: 5.010798103424051


In [3]:
#Question 9
import math, numpy as np, pandas as pd

# --- Parameters ---
S0 = 80.0
K = 80.0   # ATM strike
r = 0.055
sigma = 0.35
T = 0.25  # 3 months

# Merton jump parameters
lam = 0.25        # jump intensity λ
mu_j = -0.5       # mean of log jump size
delta = 0.228     # std dev of log jump size

# Monte Carlo parameters
Nsim = 60000      # number of simulations (increase for tighter error bars)
Nsteps = 50       # time steps
dt = T / Nsteps

# --- Precomputations ---
kappa = math.exp(mu_j + 0.5 * delta**2) - 1.0  # E[J - 1]
mu_r = r - lam * kappa - 0.5 * sigma**2        # drift adjustment
discount = math.exp(-r * T)

# --- Simulation ---
np.random.seed(20250901)   # for reproducibility
S = np.full(Nsim, S0, dtype=np.float64)

sqrt_dt = math.sqrt(dt)
lam_dt = lam * dt

for step in range(Nsteps):
    Z = np.random.normal(size=Nsim)
    diffusion = np.exp(mu_r * dt + sigma * sqrt_dt * Z)
    Nj = np.random.poisson(lam_dt, size=Nsim)  # number of jumps
    jump_mult = np.ones(Nsim, dtype=np.float64)
    idx = Nj > 0
    if np.any(idx):
        Nj_pos = Nj[idx]
        W = np.random.normal(size=Nj_pos.size)
        jump_mult[idx] = np.exp(mu_j * Nj_pos + delta * np.sqrt(Nj_pos) * W)
    S *= diffusion * jump_mult

# --- Payoffs ---
call_payoffs = np.maximum(S - K, 0.0)
put_payoffs  = np.maximum(K - S, 0.0)

call_price = discount * call_payoffs.mean()
put_price  = discount * put_payoffs.mean()

# Standard errors and confidence intervals
call_se = discount * call_payoffs.std(ddof=1) / math.sqrt(Nsim)
put_se  = discount * put_payoffs.std(ddof=1) / math.sqrt(Nsim)

z = 1.96
call_ci = (call_price - z*call_se, call_price + z*call_se)
put_ci  = (put_price  - z*put_se,  put_price  + z*put_se)

# --- Black-Scholes for comparison ---
def std_norm_cdf(x):
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

def bs_call_price(S, K, r, sigma, T):
    if T == 0: return max(S-K, 0)
    d1 = (math.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    return S * std_norm_cdf(d1) - K * math.exp(-r*T) * std_norm_cdf(d2)

def bs_put_price(S, K, r, sigma, T):
    if T == 0: return max(K-S, 0)
    d1 = (math.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    return K * math.exp(-r*T) * std_norm_cdf(-d2) - S * std_norm_cdf(-d1)

call_bs = bs_call_price(S0, K, r, sigma, T)
put_bs  = bs_put_price(S0, K, r, sigma, T)

# --- Results ---
print("Merton Model (λ=0.25)")
print(f"Call price: {call_price:.4f}  (SE={call_se:.4f}, 95% CI={call_ci})")
print(f"Put  price: {put_price:.4f}  (SE={put_se:.4f}, 95% CI={put_ci})")
print("\nBlack–Scholes (no jumps)")
print(f"Call price: {call_bs:.4f}")
print(f"Put  price: {put_bs:.4f}")


Merton Model (λ=0.25)
Call price: 6.7957  (SE=0.0410, 95% CI=(np.float64(6.715400451990504), np.float64(6.8760869040194805)))
Put  price: 5.7910  (SE=0.0383, 95% CI=(np.float64(5.715871224321395), np.float64(5.866102051144869)))

Black–Scholes (no jumps)
Call price: 6.1033
Put  price: 5.0108


In [6]:
#Question 10
import math, numpy as np, pandas as pd

# Parameters
S0 = 100.0
K = 100.0
r = 0.05
sigma = 0.20
T = 0.25
N = 50
h = 1.0  # finite difference bump for delta/gamma

# Binomial CRR pricing function
def crr_price(S0_local, K, r, sigma, T, N, is_call=True, american=False):
    dt = T / N
    u = math.exp(sigma * math.sqrt(dt))
    d = 1.0 / u
    p = (math.exp(r * dt) - d) / (u - d)
    discount = math.exp(-r * dt)

    # terminal prices
    ST = np.array([S0_local * (u**j) * (d**(N-j)) for j in range(N+1)])
    if is_call:
        values = np.maximum(ST - K, 0.0)
    else:
        values = np.maximum(K - ST, 0.0)

    # backward induction
    for i in range(N-1, -1, -1):
        values = discount * (p * values[1:i+2] + (1-p) * values[0:i+1])
        if american:
            prices = np.array([S0_local * (u**j) * (d**(i-j)) for j in range(i+1)])
            if is_call:
                values = np.maximum(values, prices - K)
            else:
                values = np.maximum(values, K - prices)
    return float(values[0])

# Central difference for Delta & Gamma
def greeks_fd(S0, K, r, sigma, T, N, is_call=True, american=False, h=1.0):
    V = crr_price(S0, K, r, sigma, T, N, is_call=is_call, american=american)
    V_up = crr_price(S0 + h, K, r, sigma, T, N, is_call=is_call, american=american)
    V_down = crr_price(S0 - h, K, r, sigma, T, N, is_call=is_call, american=american)
    delta = (V_up - V_down) / (2*h)
    gamma = (V_up - 2*V + V_down) / (h**2)
    return V, delta, gamma

# Collect results
results = []
for style in ['European','American']:
    amer = (style=='American')
    for opt in ['Call','Put']:
        is_call = (opt=='Call')
        price, delta, gamma = greeks_fd(S0, K, r, sigma, T, N, is_call, amer, h)
        results.append({
            'Style': style,
            'Option': opt,
            'Price': price,
            'Delta': delta,
            'Gamma': gamma
        })

df = pd.DataFrame(results)
print(df)


      Style Option     Price     Delta     Gamma
0  European   Call  4.595081  0.569120  0.110569
1  European    Put  3.352861 -0.430880  0.110569
2  American   Call  4.595081  0.569120  0.110569
3  American    Put  3.469215 -0.451008  0.096200


In [ ]:
#Question 14
def generate_heston_paths(S0, T, r, kappa, theta, v0, rho, xi, steps, Npaths):
    dt = T / steps
    prices = np.zeros((Npaths, steps+1))
    v_t = np.full(Npaths, v0)
    S_t = np.full(Npaths, S0)
    prices[:, 0] = S_t

    for t in range(1, steps+1):
        cov = np.array([[1, rho],[rho, 1]])
        dW = np.random.multivariate_normal([0, 0], cov, Npaths) * np.sqrt(dt)
        v_t = np.abs(v_t + kappa * (theta - v_t) * dt + xi * np.sqrt(v_t) * dW[:, 1])
        S_t = S_t * np.exp((r - 0.5 * v_t) * dt + np.sqrt(v_t) * dW[:, 0])
        prices[:, t] = S_t
    return prices

def european_up_and_in_call_price(S0, K, barrier, r, T, kappa, theta, v0, rho, xi, steps, Npaths):
    prices = generate_heston_paths(S0, T, r, kappa, theta, v0, rho, xi, steps, Npaths)
    hit_barrier = (prices >= barrier).any(axis=1)
    payoffs = np.where(hit_barrier, np.maximum(prices[:, -1] - K, 0), 0)
    discount_factor = np.exp(-r * T)
    return discount_factor * np.mean(payoffs)

S0 = 80
K = 95
barrier = 95
r = 0.055
T = 0.25
kappa = 1.85
theta = 0.045
v0 = 0.032
xi = 0.35
rho = -0.70
steps = 200
Npaths = 100000

up_and_in_call_price = european_up_and_in_call_price(S0, K, barrier, r, T, kappa, theta, v0, rho, xi, steps, Npaths)

print(f"Up-and-In Call Price: {up_and_in_call_price:.4f}")

Up-and-In Call Price: 0.0069


In [ ]:

#Question 11 and 12

import numpy as np
import pandas as pd

# ---------- Global assignment parameters ----------
S0 = 80.0
r = 0.055
T = 0.25  # 3 months
seed = 42  # fix for reproducibility

# Heston params (given + placeholder for vol-of-vol)
v0 = 0.032        # initial variance
kappa_v = 1.85    # mean reversion speed
theta_v = 0.045   # long-run variance
eta_v = 0.40      # vol-of-vol (check with Team A!)

# Merton jump params (given)
jump_mu = -0.5
jump_delta = 0.22

# Monte Carlo controls
n_sims = 20000   # increase for accuracy
n_steps = 90     # time steps


# ---------- Utility functions ----------
def round_cents(x):
    return np.round(x + 1e-12, 2)

def parity_rhs(S0, K, r, T):
    return S0 - K * np.exp(-r * T)

def parity_check(call_price, put_price, S0, K, r, T, tol=0.02):
    lhs = call_price - put_price
    rhs = parity_rhs(S0, K, r, T)
    residual = lhs - rhs
    return residual, abs(residual) <= tol


# ---------- Heston Monte Carlo ----------
def price_european_heston_mc(
    S0, K, r, T,
    v0, kappa, theta, eta, rho,
    n_sims=100_000, n_steps=252, seed=42, payoff="call"
):
    rng = np.random.default_rng(seed)
    dt = T / n_steps
    S = np.full(n_sims, S0)
    v = np.full(n_sims, v0)

    for _ in range(n_steps):
        Z1 = rng.standard_normal(n_sims)
        Z2 = rng.standard_normal(n_sims)
        W1 = Z1
        W2 = rho * Z1 + np.sqrt(1 - rho**2) * Z2

        v_plus_sqrt = np.sqrt(np.maximum(v, 0.0))
        v_plus = np.maximum(v, 0.0)

        v = v + kappa * (theta - v_plus) * dt + eta * v_plus_sqrt * np.sqrt(dt) * W2
        S = S * np.exp((r - 0.5 * v_plus) * dt + v_plus_sqrt * np.sqrt(dt) * W1)

    discount = np.exp(-r * T)
    if payoff == "call":
        payoffs = np.maximum(S - K, 0.0)
    else:
        payoffs = np.maximum(K - S, 0.0)

    est = discount * np.mean(payoffs)
    return est


# ---------- Merton Jump-Diffusion Monte Carlo ----------
def price_european_merton_mc(
    S0, K, r, T,
    sigma, lam,
    jump_mu, jump_delta,
    n_sims=100_000, n_steps=252, seed=42, payoff="call"
):
    rng = np.random.default_rng(seed)
    dt = T / n_steps
    kappa = np.exp(jump_mu + 0.5 * jump_delta**2) - 1.0

    S = np.full(n_sims, S0)
    for _ in range(n_steps):
        Z = rng.standard_normal(n_sims)
        diff = (r - lam * kappa - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z

        Nj = rng.poisson(lam * dt, size=n_sims)
        jumps = np.ones(n_sims)
        pos = Nj > 0
        if np.any(pos):
            Y_sum = rng.normal(loc=Nj[pos] * jump_mu, scale=np.sqrt(Nj[pos]) * jump_delta)
            jumps[pos] = np.exp(Y_sum)

        S = S * np.exp(diff) * jumps

    discount = np.exp(-r * T)
    if payoff == "call":
        payoffs = np.maximum(S - K, 0.0)
    else:
        payoffs = np.maximum(K - S, 0.0)

    est = discount * np.mean(payoffs)
    return est


# ---------- Q11: Put–Call Parity ----------
K_atm = S0
parity_rows = []

# Heston (rho = -0.30 and -0.70)
for rho in [-0.30, -0.70]:
    c_hat = price_european_heston_mc(S0, K_atm, r, T, v0, kappa_v, theta_v, eta_v, rho,
                                     n_sims=n_sims, n_steps=n_steps, seed=seed, payoff="call")
    p_hat = price_european_heston_mc(S0, K_atm, r, T, v0, kappa_v, theta_v, eta_v, rho,
                                     n_sims=n_sims, n_steps=n_steps, seed=seed+1, payoff="put")
    resid, ok = parity_check(c_hat, p_hat, S0, K_atm, r, T)
    parity_rows.append({
        "Model": f"Heston (rho={rho})",
        "Call ($)": round_cents(c_hat),
        "Put ($)": round_cents(p_hat),
        "C - P ($)": round_cents(c_hat - p_hat),
        "S0 - K e^{-rT} ($)": round_cents(parity_rhs(S0, K_atm, r, T)),
        "Residual ($)": round_cents(resid),
        "Parity OK?": "Yes" if ok else "No"
    })

# Merton (lambda = 0.75 and 0.25)
sigma_bs = 0.35
for lam in [0.75, 0.25]:
    c_hat = price_european_merton_mc(S0, K_atm, r, T, sigma=sigma_bs, lam=lam,
                                     jump_mu=jump_mu, jump_delta=jump_delta,
                                     n_sims=n_sims, n_steps=n_steps, seed=seed, payoff="call")
    p_hat = price_european_merton_mc(S0, K_atm, r, T, sigma=sigma_bs, lam=lam,
                                     jump_mu=jump_mu, jump_delta=jump_delta,
                                     n_sims=n_sims, n_steps=n_steps, seed=seed+1, payoff="put")
    resid, ok = parity_check(c_hat, p_hat, S0, K_atm, r, T)
    parity_rows.append({
        "Model": f"Merton (lambda={lam})",
        "Call ($)": round_cents(c_hat),
        "Put ($)": round_cents(p_hat),
        "C - P ($)": round_cents(c_hat - p_hat),
        "S0 - K e^{-rT} ($)": round_cents(parity_rhs(S0, K_atm, r, T)),
        "Residual ($)": round_cents(resid),
        "Parity OK?": "Yes" if ok else "No"
    })

parity_df = pd.DataFrame(parity_rows)
print("Q11 — Put–Call Parity Check")
print(parity_df)


# ---------- Q12: Strike Sweep ----------
moneyness_targets = [0.85, 0.90, 0.95, 1.00, 1.05, 1.10, 1.15]
strikes = [S0 / m for m in moneyness_targets]

sweep_rows = []
rho_for_sweep = -0.70
lam_for_sweep = 0.75

for K in strikes:
    c_h = price_european_heston_mc(S0, K, r, T, v0, kappa_v, theta_v, eta_v, rho_for_sweep,
                                   n_sims=n_sims, n_steps=n_steps, seed=seed, payoff="call")
    c_m = price_european_merton_mc(S0, K, r, T, sigma=sigma_bs, lam=lam_for_sweep,
                                   jump_mu=jump_mu, jump_delta=jump_delta,
                                   n_sims=n_sims, n_steps=n_steps, seed=seed, payoff="call")
    sweep_rows.append({
        "Strike K": round_cents(K),
        "Moneyness (S0/K)": np.round(S0 / K, 3),
        "Call_Heston ($)": round_cents(c_h),
        "Call_Merton ($)": round_cents(c_m),
    })

sweep_df = pd.DataFrame(sweep_rows).sort_values("Strike K")
print("\nQ12 — Strike Sweep")
print(sweep_df)

Q11 — Put–Call Parity Check
                  Model  Call ($)  Put ($)  C - P ($)  S0 - K e^{-rT} ($)  \
0     Heston (rho=-0.3)      3.49     2.35       1.15                1.09   
1     Heston (rho=-0.7)      3.51     2.37       1.15                1.09   
2  Merton (lambda=0.75)      8.37     7.14       1.23                1.09   
3  Merton (lambda=0.25)      6.91     5.72       1.19                1.09   

   Residual ($) Parity OK?  
0          0.06         No  
1          0.05         No  
2          0.14         No  
3          0.10         No  

Q12 — Strike Sweep
   Strike K  Moneyness (S0/K)  Call_Heston ($)  Call_Merton ($)
6     69.57              1.15            11.75            15.17
5     72.73              1.10             8.94            12.89
4     76.19              1.05             6.12            10.61
3     80.00              1.00             3.51             8.37
2     84.21              0.95             1.47             6.27
1     88.89              0.90        

In [7]:
#Question 15
import math, numpy as np

# --------------------------
# Parameters (Merton model)
# --------------------------
S0 = 80.0
K = 65.0        # strike
H = 65.0        # down-and-in barrier = strike
r = 0.055
sigma = 0.35
T = 0.25        # 3 months

# Merton jump params (as in Question 8)
lam = 0.75      # jump intensity
mu_j = -0.5     # mean of log jump size
delta = 0.228   # std dev of log jump size

# Monte Carlo settings
Nsim = 100000   # number of simulated paths
Nsteps = 100    # time steps for path monitoring
dt = T / Nsteps

# Precompute drift adjustment
kappa = math.exp(mu_j + 0.5 * delta**2) - 1.0
mu_r = r - lam * kappa - 0.5 * sigma**2  # diffusion part drift under RN measure

discount = math.exp(-r * T)

# Seed for reproducibility
np.random.seed(20250902)

# Initialize arrays
S = np.full(Nsim, S0, dtype=np.float64)
hit = np.zeros(Nsim, dtype=bool)   # whether path has hit barrier at any monitored step

sqrt_dt = math.sqrt(dt)
lam_dt = lam * dt

# simulate forward in time
for step in range(Nsteps):
    # diffusion (Euler-like exact lognormal increment for diffusion part)
    Z = np.random.normal(size=Nsim)
    diffusion = np.exp(mu_r * dt + sigma * sqrt_dt * Z)
    # jumps: Poisson count per small interval
    Nj = np.random.poisson(lam_dt, size=Nsim)
    jump_mult = np.ones(Nsim, dtype=np.float64)
    idx = Nj > 0
    if np.any(idx):
        Nj_pos = Nj[idx]
        W = np.random.normal(size=Nj_pos.size)
        # product of Nj iid lognormal jumps has log mean = mu_j * Nj_pos, variance = delta^2 * Nj_pos
        jump_mult[idx] = np.exp(mu_j * Nj_pos + delta * np.sqrt(Nj_pos) * W)
    # update stock
    S *= diffusion * jump_mult
    # update barrier hitting (down-and-in)
    hit = hit | (S <= H)

# Terminal payoffs
# PDI: only pays if barrier was hit at some time; payoff = max(K - S_T, 0)
payoffs_pdi = np.where(hit, np.maximum(K - S, 0.0), 0.0)
pdi_price = discount * payoffs_pdi.mean()
pdi_se = discount * payoffs_pdi.std(ddof=1) / math.sqrt(Nsim)

# For comparison: plain European put with same K under Merton (no barrier)
payoffs_put = np.maximum(K - S, 0.0)
put_price = discount * payoffs_put.mean()
put_se = discount * payoffs_put.std(ddof=1) / math.sqrt(Nsim)

# Black-Scholes put for K=65 (no jumps) for reference
from math import erf
def std_norm_cdf(x): return 0.5*(1 + erf(x/math.sqrt(2.0)))
def bs_put(S,K,r,sigma,T):
    if T==0: return max(K-S,0)
    d1 = (math.log(S/K) + (r + 0.5*sigma*sigma)*T) / (sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    return K*math.exp(-r*T) * std_norm_cdf(-d2) - S*std_norm_cdf(-d1)

bs_put_price = bs_put(S0, K, r, sigma, T)

# Print results
print("Merton jump-diffusion (lambda=0.75)")
print(f"PDI (down-and-in put, H=K=65): price = {pdi_price:.6f}, SE = {pdi_se:.6f}")
print(f"European put (same K=65):        price = {put_price:.6f}, SE = {put_se:.6f}")
print()
print("Black-Scholes (no jumps) reference:")
print(f"European put (K=65) BS price = {bs_put_price:.6f}")


Merton jump-diffusion (lambda=0.75)
PDI (down-and-in put, H=K=65): price = 2.765133, SE = 0.024091
European put (same K=65):        price = 2.765133, SE = 0.024091

Black-Scholes (no jumps) reference:
European put (K=65) BS price = 0.612683
